In [ ]:
from Bio import SeqIO
import pandas as pd
import gffutils

In [ ]:
# Genome
for record in SeqIO.parse(snakemake.input[0], "fasta"):
    print(record.id)

In [ ]:
genome = SeqIO.to_dict(SeqIO.parse(snakemake.input[0], "fasta"))

In [ ]:
# Annotation

db = gffutils.FeatureDB(snakemake.input[1])

db

In [ ]:
list(db.featuretypes())

In [ ]:
list(db.features_of_type("gene"))

In [ ]:
for row in db.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(row['name'])

In [ ]:
for row in db.execute("pragma table_info(features);"):
    print(row['name'])

In [ ]:
for row in db.execute("select id from features where id like '%AT2G19110%';"):
    print(row['id'])

In [ ]:
list(db['gene-AT2G19110'].attributes)

In [ ]:
hma4_annotation = db['gene-AT2G19110']

In [ ]:
hma4_annotation.seqid

In [ ]:
genome[hma4_annotation.seqid][hma4_annotation.start-1:hma4_annotation.end].seq

In [ ]:
# start coding here

annotation = gffutils.FeatureDB(snakemake.input[1])

genome = SeqIO.to_dict(SeqIO.parse(snakemake.input[0], "fasta"))

gene_annotation_id = f"gene-{snakemake.wildcards[0].split('.')[0]}"

gene_annotation = annotation[gene_annotation_id]

gene_sequence = genome[gene_annotation.seqid][gene_annotation.start-1:gene_annotation.end]

df = pd.read_csv(snakemake.input[2])

df

In [ ]:
def get_substitution_dna_window(variant, gene_sequence, gene_annotation):
    in_gene_position = variant["position"] - gene_annotation.start

    ref, alt = parse_dna_substitution(variant["amino acid change"])

    assert ref == gene_sequence[in_gene_position].upper()
    
    return str(gene_sequence[in_gene_position - 256 : in_gene_position + 256].seq)

In [ ]:
from Bio.SeqUtils import seq1
import re


def parse_aa_substitution(HGVS_string):
    prot_match = re.match(r"p\.([A-Z]{1}[a-z]{2})\d+([A-Z]{1}[a-z]{2})", HGVS_string)
    reference, alternative = prot_match.groups()
    return seq1(reference), seq1(alternative)


def parse_dna_substitution(HGVS_string):
    dna_match = re.search(r"c\..*([ATCG])>([ATCG])$", HGVS_string)
    reference, alternative = dna_match.groups()
    return reference, alternative


In [ ]:
def add_sequence_window(snp_effect_path, reference_fasta_path, annotation_db_path):
    annotation = gffutils.FeatureDB(annotation_db_path)
    genome = SeqIO.to_dict(SeqIO.parse(reference_fasta_path, "fasta"))
    
    gene_annotation_id = f"gene-{snakemake.wildcards[0].split('.')[0]}"
    gene_annotation = annotation[gene_annotation_id]
    
    gene_sequence = genome[gene_annotation.seqid][gene_annotation.start-1:gene_annotation.end]
    
    df = pd.read_csv(snp_effect_path)

    df = df.dropna(subset=["amino acid change"])
    df = df[ ~ df["amino acid change"].str.contains("ins", na=False) ]
    df = df[ ~ df["amino acid change"].str.contains("del", na=False) ]


    df["sequence_window"] = df.apply(
        lambda x: get_substitution_dna_window(x, gene_sequence, gene_annotation), axis=1
    )
    return df


In [ ]:
df = add_sequence_window(snakemake.input[2], snakemake.input[0], snakemake.input[1])

df

In [ ]:
df[df["sequence_window"] == "-"]

In [ ]:
gene_sequence

In [ ]:
pos = 8279002 - gene_annotation.start
pos

In [ ]:
gene_sequence[pos]

In [ ]:
import numpy as np


def add_surprise_bonus(df):
    # Entropy H in base-4
    df["H"] = -(
        df[["p_a", "p_c", "p_g", "p_t"]]
        .pipe(lambda p: p * (np.log2(p) / 2))
        .sum(axis=1)
    )

    # Surprisal I in base-4
    df["I"] = -(np.log2(df["p_obs"]) / 2)
    df["surprise bonus"] = df["H"] - df["I"]


In [ ]:
test = pd.DataFrame({
    "p_a": [0.3],
    "p_c": [0.3],
    "p_g": [0.3],
    "p_t": [0.1],
    "p_obs": [0.1], 
})
test

In [ ]:
add_surprise_bonus(test)

test

In [ ]:
test = pd.DataFrame({
    "p_a": [0.4],
    "p_c": [0.4],
    "p_g": [0.1],
    "p_t": [0.1],
    "p_obs": [0.1], 
    "p_ref": [0.1], 
})
test

In [ ]:
add_surprise_bonus(test)

test

In [ ]:
gpn = np.log2(test[["p_a", "p_c", "p_g", "p_t"]] / test["p_ref"].to_numpy()[:, None])

gpn
        